In [2]:
#Import functions to use for this analysis
import pandas as pd
import numpy as np
import time
import datetime as dt

In [3]:
#Import csv
df_load = pd.read_csv('~/Documents/swimdata.csv')

In [4]:
#Look at first rows of dataframe similar to R head() function 
df_load[0:6]

,Place,Bib,Name,First name,Last name,Team name,Team name 2,Distance,Category,Age,Gender,Time,Difference,% Back,% Winning,% Average
0,1,250.0,Jude Williams,Jude,Williams,Mentone CA,Non-Wetsuit,1 Mile,M1517,15.0,Male,19:15.7,-,-,100%,40.14%
1,2,2.0,Steve Allnutt,Steve,Allnutt,San Diego CA,Non-Wetsuit,1 Mile,M2529,28.0,Male,19:42.6,+0:26.9,+2.33%,97.73%,38.75%
2,3,272.0,Sandoval Martha,Sandoval,Martha,Mexicali Baja California,Non-Wetsuit,1 Mile,F1824,19.0,Female,20:02.3,+0:46.6,+4.03%,96.12%,37.73%
3,4,48.0,Daniel Escalante,Daniel,Escalante,Mexicali Baja California,Non-Wetsuit,1 Mile,M1517,15.0,Male,20:29.3,+1:13.6,+6.37%,94.01%,36.33%
4,5,134.0,CLAUDIA MACIAS,CLAUDIA,MACIAS,Mexicali Baja California,Non-Wetsuit,1 Mile,F1517,17.0,Female,20:29.4,+1:13.7,+6.38%,94.01%,36.33%
5,6,23.0,EMILIO CAMARILLO,EMILIO,CAMARILLO,Chula Vista CA,Non-Wetsuit,1 Mile,M1517,16.0,Male,20:33.9,+1:18.2,+6.77%,93.66%,36.09%


In [5]:
#Look at last rows not inclusive of index
df_load[-4:-1]

,Place,Bib,Name,First name,Last name,Team name,Team name 2,Distance,Category,Age,Gender,Time,Difference,% Back,% Winning,% Average
273,-,233.0,MICHAEL SULLIVAN,MICHAEL,SULLIVAN,San Diego CA,Wetsuit,1 Mile,M2529,28.0,Male,DNS,-,-,-,-
274,-,238.0,Lori Thompson,Lori,Thompson,Trabuco Canyon CA,Wetsuit,1 Mile,F4044,40.0,Female,DNS,-,-,-,-
275,-,245.0,Mauricio Vega Kuri Garcia,Mauricio,Vega Kuri Garcia,Calexico CA,Non-Wetsuit,1 Mile,M4549,45.0,Male,DNS,-,-,-,-


In [11]:
#Generate new dataframe with only relevant columns
df = df_load[['Place','Name','Category','Gender','Time']].iloc[0:275]

In [12]:
#Count time but also get median and standard deviation
participant_count = df.groupby(['Category']).count()['Time']

In [13]:
#Using logical operator on DNS, get percent of people by category that finished race
finish_percent = df[df.Time <> 'DNS'].groupby(['Category']).count()['Time']/df.groupby(['Category']).count()['Time']

In [14]:
cat_summary = pd.concat([finish_percent,participant_count],axis=1)
cat_summary.columns = ['Finish Rate','Participant Count'] #Add median and standard deviation of time
cat_summary

,Finish Rate,Participant Count
Category,,
F1112,1.000000,1
F1314,1.000000,14
F1517,1.000000,9
F1824,1.000000,15
F2529,0.923077,13
F3034,1.000000,11
F3539,1.000000,4
F4044,0.900000,10
F4549,0.857143,7


In [15]:
#Turn time from timestamp format into array form to get total seconds
df['TimeArray'] = df['Time'].str.split(':',1)

In [16]:
#Turn array into seconds
minutes_list = [item[0] for item in df['TimeArray']]
seconds_list = [item[-1] for item in df['TimeArray']]

df['minutelist'] = pd.Series(minutes_list, index=df.index)
df['secondslist'] = pd.Series(seconds_list, index=df.index)

#Bad option:
#np.where(df['Time'] <> 'DNS', 60*int(df['minutelist'])+int(df['secondslist']), 'DNS')

df['time_seconds'] = pd.to_numeric(df['minutelist'], errors='coerce')*60 + pd.to_numeric(df['secondslist'], errors='coerce')

In [48]:
#Look at median and variance of times by age/gender category
df_std = df.groupby('Category')['time_seconds'].agg(['median','std'])

In [49]:
#See median and standard deviation in minutes (with decimal) instead of seconds
df_std[['median','std']].div(60)

,median,std
Category,,
F1112,27.123333,NaN
F1314,24.980000,7.369894
F1517,20.860000,2.577782
F1824,27.965000,5.055897
F2529,31.280000,6.091010
F3034,37.990000,5.550781
F3539,31.170000,2.798447
F4044,36.088333,8.459639
F4549,34.066667,9.746258


In [40]:
participant_locations = df_load['Team name'].iloc[0:275]

In [42]:
#Get geo into dataframe
df['City'] = df_load['Team name']

In [46]:
#Cut into Baja California, San Diego, other CA, other
df['geo_group'] = pd.np.where(df.City.str.contains("Mexicali Baja California"), "Baja Cali",
                   pd.np.where(df.City.str.contains("San Diego CA"), "San Diego",
                   pd.np.where(df.City.str.contains("CA"), "California", "Other")))

In [55]:
#Look at median and variance of times by geography of participant
df_geo = df.groupby('geo_group')['time_seconds'].agg(['median','std','count'])
df_geo

,median,std,count
geo_group,,,
Baja Cali,1294.7,97.010225,19
California,1963.0,447.675721,137
Other,1835.2,436.257225,43
San Diego,1879.9,406.759422,58


In [56]:
df_geo[['median','std']].div(60)

,median,std
geo_group,,
Baja Cali,21.578333,1.616837
California,32.716667,7.461262
Other,30.586667,7.270954
San Diego,31.331667,6.779324
